# MNIST認識のシンプルなコード

## 必要なパッケージをインポート

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import time

## グローバル定数の設定

In [2]:
batch_size = 200                # ミニバッチサイズ
device = torch.device('cpu')
sgd_lr = 0.1

## データローダの準備

In [3]:
root = '.' # mnistデータの置き場所
download = True
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=root, train=True, transform=trans, download=download)
test_set = dset.MNIST(root=root, train=False, transform=trans)
# ローダの準備
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

## モデルの定義


In [4]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__() 
        self.l1 = nn.Linear(784, 784)
        self.l2 = nn.Linear(784, 784)
        self.l3 = nn.Linear(784, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return F.log_softmax(x, dim=1)

## 訓練ループ

In [10]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=sgd_lr)
running_loss = 0.0
i = 0
for loop in range(10):
    for (input, target) in train_loader:
        i = i + 1
        input, target = input.to(device), target.to(device)
        input = input.view(-1, 28*28) # テンソルのサイズを整える必要がある
        optimizer.zero_grad()    # optimizerの初期化
        output = model(input)     # 推論計算
        loss = F.nll_loss(output, target) # 損失関数の定義
        loss.backward()             # バックプロパゲーション
        optimizer.step()            # パラメータ更新
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%5d] loss: %.3f' %
                  (i + 1, running_loss / 100))
            running_loss = 0.0

[  100] loss: 1.428
[  200] loss: 0.536
[  300] loss: 0.407
[  400] loss: 0.346
[  500] loss: 0.306
[  600] loss: 0.284
[  700] loss: 0.259
[  800] loss: 0.251
[  900] loss: 0.229
[ 1000] loss: 0.217
[ 1100] loss: 0.195
[ 1200] loss: 0.192
[ 1300] loss: 0.181
[ 1400] loss: 0.161
[ 1500] loss: 0.166
[ 1600] loss: 0.156
[ 1700] loss: 0.143
[ 1800] loss: 0.138
[ 1900] loss: 0.133
[ 2000] loss: 0.130
[ 2100] loss: 0.124
[ 2200] loss: 0.120
[ 2300] loss: 0.113
[ 2400] loss: 0.112
[ 2500] loss: 0.100
[ 2600] loss: 0.102
[ 2700] loss: 0.101
[ 2800] loss: 0.094
[ 2900] loss: 0.091
[ 3000] loss: 0.091


## Accuracyの評価

In [11]:
correct =  0
count = 0
for (input, target) in test_loader:
    input, target = input.to(device), target.to(device)
    input = input.view(-1, 28*28)
    output = model(input)     # 推論計算

    pred = output.data.max(1)[1]
    correct += pred.eq(target.data).sum()
    count += batch_size

print ('accuracy = ', float(correct)/float(count))

accuracy =  0.9703
